In [1]:
import tkinter as tk
from tkinter import ttk as ttk
import tkinter.filedialog as fl
import tkinter.messagebox as mb
import numpy as np
import openpyxl as opxl
import pandas as pd
import os
import pandas.io.formats.excel
import re

root = tk.Tk()
root.title("データ統合")
root.geometry("410x230+500+500")


#-----------------------------------------------------------------------------------------------------------
#関数 flg_header  ラジオボタンからTrue,Falseを取得　保存ファイルのヘッダーのon,offに利用 初期値：True
#-----------------------------------------------------------------------------------------------------------
def flg_header():
    global flg_head
    flg_head = fl_h.get()
    # print(flg_head)
    return flg_head


#-----------------------------------------------------------------------------------------------------------
#関数 csv_convert　csvファイルをxlsxファイルへ変換
#-----------------------------------------------------------------------------------------------------------
def csv_convert():
    #下段のファイル名初期化　クリップボード初期化
    wr_filename.delete(0, tk.END)
    root.clipboard_clear()
    #ヘッダー用関数
    flg_head = flg_header()
    #ファイル読み込みダイアログ　単数選択
    filetype=[("data files", "*.csv")]
    filepath = fl.askopenfilename(initialdir=os.getcwd(), filetypes=filetype, title="csvファイルを選択")
    f_num = len(filepath)
    #ファイルパスからファイル名取得
    filename = os.path.basename(filepath)
    #元の名前のまま拡張子を変更
    convert_fn = filename.replace(".csv", ".xlsx")
    #ファイル保存ダイアログ　単数選択 ファイル選択したときに次の処理へ キャンセル時は何もしない
    if f_num != 0:
        root.filename = fl.asksaveasfilename(title = "名前を付けて保存", filetypes = [("Excel", "*.xlsx")],
        initialdir = os.getcwd(), initialfile = convert_fn, defaultextension = ".xlsx") # ファイルフィルタ
        save_f_p = root.filename #ファイルのパス+ファイル名
    else:
        return
    #保存を押したら、実行　キャンセル時は何もしない
    if save_f_p != "":
        #utf-8で読み込み
        try:
            df = pd.read_csv(filepath, encoding="utf-8")
            sheetname = filename.replace(".csv", "")                                        #シート名用に拡張子削除
            with pd.ExcelWriter(save_f_p, mode="w", engine="openpyxl") as writer:
                pd.io.formats.excel.ExcelFormatter.header_style = None                      #インデックス、カラム強調表示off
                df.to_excel(writer, sheet_name=sheetname, index=False, header=flg_head)     #headerはラジオボタンでon,off
            save_name = os.path.basename(save_f_p)
            print(f"保存ファイル名：{save_name}")
            wr_filename.insert(tk.END,save_name)
            mb.showinfo("csvデータ変換", "完了")
        #utf-8が失敗した時は、shift-jisで読み込み
        except UnicodeDecodeError as e:
            print(e, "from_csv_convert")
            df = pd.read_csv(filepath, encoding="shift-jis")
            sheetname = filename.replace(".csv", "")
            with pd.ExcelWriter(save_f_p, mode="w", engine="openpyxl") as writer:
                pd.io.formats.excel.ExcelFormatter.header_style = None
                df.to_excel(writer, sheet_name=sheetname, index=False, header=flg_head)
            save_name = os.path.basename(save_f_p)
            print(f"保存ファイル名：{save_name}")
            wr_filename.insert(tk.END,save_name)
            mb.showinfo("csvデータ変換", "完了")
    else:
        return


#-----------------------------------------------------------------------------------------------------------
#関数 csv_choice_open　csvの複数ファイルをxlsxファイルへ変換(各Sheet_tabに1データずつを入れる)
#-----------------------------------------------------------------------------------------------------------
def csv_choice_open():
    global filepath, f_num, flg_head
    #下段のファイル名初期化　クリップボード初期化
    wr_filename.delete(0, tk.END)
    root.clipboard_clear()
    flg_head = flg_header()
    #ファイル読み込みダイアログ　複数選択
    filetype=[("data files", "*.csv")]
    filepath = fl.askopenfilenames(initialdir=os.getcwd(), filetypes=filetype, title="csvファイルを選択")
    f_num = len(filepath)
    #複数ファイルを選択するように1個は弾く
    if f_num == 1:
        mb.showinfo("エラー", "複数のデータを選択してください")
    #キャンセル時は何もしない
    elif f_num == 0:
        return
    else:
        save2xlsx()


#-----------------------------------------------------------------------------------------------------------
#関数 save2xlsx　xlsxファイルへの保存ダイアログ表示
#-----------------------------------------------------------------------------------------------------------
def save2xlsx():
    global save_f_p
    #保存ファイル名ダイアログ
    root.filename = fl.asksaveasfilename(title = "名前を付けて保存", filetypes = [("Excel", "*.xlsx")],
    initialdir = os.getcwd(), defaultextension = ".xlsx") # ファイルフィルタ
    save_f_p = root.filename
    #保存時、キャンセル押すと何もしない
    if save_f_p == "":
        return
    else:
        csv2xlsx_tab()


#-----------------------------------------------------------------------------------------------------------
#関数 csv2xlsx_tab　csvの複数ファイルから1つのxlsxファイル変換への実行処理　
#-----------------------------------------------------------------------------------------------------------
def csv2xlsx_tab():
    #1シート目に最初のCSVファイルの内容を書き込み　最初のファイルはwモード EXCELのシート名が[Sheet]からファイル名にするため
    for i in range(f_num):
        filename = os.path.basename(filepath[i])
        sheetname = filename.replace(".csv", "").replace(".xlsx", "")
        # print(filename)
        #elseからスタート if os.path.existsで既存のファイルがあるかチェック
        if os.path.exists(save_f_p):
            with pd.ExcelWriter(save_f_p, mode="a", engine="openpyxl") as writer:
                #utf-8で読み込み
                try:
                    df = pd.read_csv(filepath[i])
                    pd.io.formats.excel.ExcelFormatter.header_style = None
                    df.to_excel(writer, sheet_name=sheetname, index=False, header=flg_head)
                #utf-8が失敗した時は、shift-jisで読み込み
                except UnicodeDecodeError as e:
                    print(e, "from_csv2xlsx_tab")
                    df = pd.read_csv(filepath[i], encoding="shift-jis")
                    pd.io.formats.excel.ExcelFormatter.header_style = None
                    df.to_excel(writer, sheet_name=sheetname, index=False, header=flg_head)

        else:
            with pd.ExcelWriter(save_f_p, engine="openpyxl") as writer:
                try:
                    df = pd.read_csv(filepath[i])
                    pd.io.formats.excel.ExcelFormatter.header_style = None
                    df.to_excel(writer, sheet_name=sheetname, index=False, header=flg_head)
                except UnicodeDecodeError as e:
                    print(e, "from_csv2xlsx_tab")
                    df = pd.read_csv(filepath[i], encoding="shift-jis")
                    pd.io.formats.excel.ExcelFormatter.header_style = None
                    df.to_excel(writer, sheet_name=sheetname, index=False, header=flg_head)

    save_name = os.path.basename(save_f_p)
    print(f"保存ファイル名：{save_name}")
    wr_filename.insert(tk.END,save_name)
    mb.showinfo("完了", "csvデータ結合完了")


#-----------------------------------------------------------------------------------------------------------
#関数 csv_choice_open　csvの複数ファイルをxlsxファイルへ変換(各Sheet_tabに1データずつを入れる)
#-----------------------------------------------------------------------------------------------------------
def csv_concat_choice_open():
    global filepath, f_num, flg_head
    #下段のファイル名初期化　クリップボード初期化
    wr_filename.delete(0, tk.END)
    root.clipboard_clear()
    flg_head = flg_header()
    #ファイル読み込みダイアログ　複数選択
    filetype=[("data files", "*.csv")]
    filepath = fl.askopenfilenames(initialdir=os.getcwd(), filetypes=filetype, title="csvファイルを選択")
    f_num = len(filepath)
    #複数ファイルを選択するように1個は弾く
    if f_num == 1:
        mb.showinfo("エラー", "複数のデータを選択してください")
    #キャンセル時は何もしない
    elif f_num == 0:
        return
    else:
        save2concat_xlsx()


#-----------------------------------------------------------------------------------------------------------
#関数 save2xlsx　xlsxファイルへの保存ダイアログ表示
#-----------------------------------------------------------------------------------------------------------
def save2concat_xlsx():
    global save_f_p
    #保存ファイル名ダイアログ
    root.filename = fl.asksaveasfilename(title = "名前を付けて保存", filetypes = [("Excel", "*.xlsx")],
    initialdir = os.getcwd(), defaultextension = ".xlsx") # ファイルフィルタ
    save_f_p = root.filename
    #保存時、キャンセル押すと何もしない
    if save_f_p == "":
        return
    else:
        csv2concat_xlsx()


#-----------------------------------------------------------------------------------------------------------
#関数 csv2xlsx_tab　csvの複数ファイルから1つのxlsxファイル変換への実行処理　
#-----------------------------------------------------------------------------------------------------------
def csv2concat_xlsx():
    #1シート目に最初のCSVファイルの内容を書き込み　最初のファイルはwモード EXCELのシート名が[Sheet]からファイル名にするため
    df_list = []
    for i in range(f_num):
        filename = os.path.basename(filepath[i])
        fn = filename.replace(".csv", "").replace(".xlsx", "")
        # print(filename)
        #新規保存ならelseからスタート if os.path.existsで既存のファイルがあるかチェック
        with pd.ExcelWriter(save_f_p, engine="openpyxl") as writer:
            #utf-8で読み込み
            try:
                df = pd.read_csv(filepath[i])
                df["ファイル名"] = fn
                df_list.append(df)
            #utf-8が失敗した時は、shift-jisで読み込み
            except UnicodeDecodeError as e:
                print(e, "from_csv2concat_xlsx")
                df = pd.read_csv(filepath[i], encoding="shift-jis")
                df["ファイル名"] = fn
                df_list.append(df)
            #リストに入れたDFをconcatで行方向に結合
            df_con = pd.concat(df_list, ignore_index=True, axis=0)
            pd.io.formats.excel.ExcelFormatter.header_style = None
            df_con.to_excel(writer, sheet_name="csv_concat", index=False, header=flg_head)

    save_name = os.path.basename(save_f_p)
    print(f"保存ファイル名：{save_name}")
    wr_filename.insert(tk.END,save_name)
    mb.showinfo("完了", "csvデータ結合完了")


#-----------------------------------------------------------------------------------------------------------
#関数 xlsx_choice_open　xlsxの複数ファイルをxlsxファイルへ変換(各Sheet_tabに1データずつ入れる)
#-----------------------------------------------------------------------------------------------------------
def xlsx_choice_open():
    global filepath, f_num, flg_head
    #下段のファイル名初期化　クリップボード初期化
    wr_filename.delete(0, tk.END)
    root.clipboard_clear()
    flg_head = flg_header()
    #ファイル読み込みダイアログ　複数選択
    filetype = [("data files", "*.xlsx")]
    filepath = fl.askopenfilenames(initialdir=os.getcwd(), filetypes=filetype, title="xlsxファイルを選択")
    f_num = len(filepath)
    #複数ファイルを選択するように1個は弾く
    if f_num == 1:
        mb.showinfo("エラー", "複数のデータを選択してください")
    #キャンセル時は何もしない
    elif f_num == 0:
        return
    else:
        xlsx2xlsx()


#-----------------------------------------------------------------------------------------------------------
#関数 xlsx2xlsx　xlsxファイルへの保存ダイアログ表示
#-----------------------------------------------------------------------------------------------------------
def xlsx2xlsx():
    global save_f_p
    #保存ファイル名ダイアログ
    root.filename = fl.asksaveasfilename(title = "名前を付けて保存", filetypes = [("Excel", "*.xlsx")],
    initialdir = os.getcwd(), defaultextension = ".xlsx") # ファイルフィルタ
    save_f_p = root.filename
    #保存時、キャンセル押すと何もしない
    if save_f_p == "":
        return
    else:
        xlsx2_tab()


#-----------------------------------------------------------------------------------------------------------
#関数 xlsx2_tab　xlsxの複数ファイルから1つのxlsxファイルへの変換実行処理
#-----------------------------------------------------------------------------------------------------------
def xlsx2_tab():
    for i in range(f_num):
        filename = os.path.basename(filepath[i])
        sheetname = filename.replace(".xlsx", "")
        # print(filename)
        #elseからスタートする　if os.path.existsで既存のファイルがあるかチェック
        if os.path.exists(save_f_p):
            with pd.ExcelWriter(save_f_p, mode="a", engine="openpyxl") as writer:
                try:
                    df = pd.read_excel(filepath[i])
                    df.columns = df.columns.str.replace(r"Unnamed: \d+", "", regex=True)        #pandasで読み込むとできるUnnamed:~を消す
                    pd.io.formats.excel.ExcelFormatter.header_style = None
                    df.to_excel(writer, sheet_name=sheetname, index=False, header=flg_head)     #headerはラジオボタンでon,off
                except UnicodeDecodeError as e:
                    print(e, "from_xlsx2_tab")
                    df = pd.read_excel(filepath[i], encoding="utf-8")
                    df.columns = df.columns.str.replace(r"Unnamed: \d+", "", regex=True)
                    pd.io.formats.excel.ExcelFormatter.header_style = None
                    df.to_excel(writer, sheet_name=sheetname, index=False, header=flg_head)
        else:
            with pd.ExcelWriter(save_f_p, engine="openpyxl") as writer:
                try:
                    df = pd.read_excel(filepath[i])
                    df.columns = df.columns.str.replace(r"Unnamed: \d+", "", regex=True)
                    pd.io.formats.excel.ExcelFormatter.header_style = None
                    df.to_excel(writer, sheet_name=sheetname, index=False, header=flg_head)
                except UnicodeDecodeError as e:
                    print(e, "from_xlsx2_tab")
                    df = pd.read_excel(filepath[i], encoding="utf-8")
                    df.columns = df.columns.str.replace(r"Unnamed: \d+", "", regex=True)
                    pd.io.formats.excel.ExcelFormatter.header_style = None
                    df.to_excel(writer, sheet_name=sheetname, index=False, header=flg_head)

    save_name = os.path.basename(save_f_p)
    print(f"保存ファイル名：{save_name}")
    wr_filename.insert(tk.END,save_name)
    mb.showinfo("完了", "エクセルデータ結合完了")


#-----------------------------------------------------------------------------------------------------------
#関数 fn_copy tk.Entryに保存したファイル名を入れて、clipborad_appendでファイル名をクリップボードに取得
#-----------------------------------------------------------------------------------------------------------
def fn_copy():
    #初期化してからコピー
    root.clipboard_clear()
    fn = wr_filename.get()
    root.clipboard_append(fn)


#------------------------------------------------------------------------------------------------------------------------------------------------
#Frame配置

frame1 = tk.Frame(root)
frame1.pack(fill=tk.X) #TOPに配置

frame2 = tk.Frame(root, width=410, height=65)
frame2.pack(side=tk.BOTTOM, anchor = tk.NW)

frame3 = tk.Frame(root, width=340, height=150)
frame3.pack(side=tk.LEFT, anchor = tk.NW) #左に配置　アンカーで左上に寄せる

frame4 = tk.Frame(root, width=130, height=150)
frame4.pack(side=tk.RIGHT, anchor = tk.NW)

#------------------------------------------------------------------------------------------------------------------------------------------------
# ラジオボタン 保存ファイルのheaderのon,off用

fl_h = tk.BooleanVar() # BooleanでTrue,Falseを取得
fl_h.set(1) #デフォルト:True
# ラジオボタン作成
rdo1 = tk.Radiobutton(frame4, value=1, variable=fl_h, text="ヘッダーあり") #True
rdo1.place(x=17, y=30)

rdo2 = tk.Radiobutton(frame4, value=0, variable=fl_h, text="ヘッダーなし") #False
rdo2.place(x=17, y=60)

# #デバッグ用
# btn = tk.Button(frame4, text="Booleanテスト", relief=tk.GROOVE, command = flg_header)
# btn.place(x=20, y=90)

#------------------------------------------------------------------------------------------------------------------------------------------------
#フォント設定

fonts = ("BIZ UDPゴシック","10", "bold")
fonts_nor = ("BIZ UDPゴシック","9")

#------------------------------------------------------------------------------------------------------------------------------------------------
#ボタン

csv_convert_btn = tk.Button(frame3, text="csv to\n\nEXCEL変換",
                            font=fonts_nor, height=4, width=13, relief=tk.GROOVE, command = csv_convert)
csv_convert_btn.grid(row = 0, column = 0, padx=15, pady=5)

csv_concat_btn = tk.Button(frame3, text="csv複数ファイル\n\nEXECL 行結合",
                           font=fonts_nor, height=4, width=13, relief=tk.GROOVE, command = csv_concat_choice_open)
csv_concat_btn.grid(row = 0, column = 1, padx=0, pady=5)


csv_tab_btn = tk.Button(frame3, text="csv複数ファイル\n\nEXECL tab変換",
                        font=fonts_nor, height=4, width=13, relief=tk.GROOVE, command = csv_choice_open) 
csv_tab_btn.grid(row = 1, column = 0, padx=15, pady=5)

xlsx_tab_btn = tk.Button(frame3, text="xlsx複数ファイル\n\nEXECL tab変換",
                         font=fonts_nor, height=4, width=13, relief=tk.GROOVE, command = xlsx_choice_open)
xlsx_tab_btn.grid(row = 1, column = 1, padx=0, pady=5)

fn_copy_btn = tk.Button(frame2, text="ファイル名コピー", relief=tk.GROOVE, command = fn_copy)
fn_copy_btn.place(x=305, y=22)

#------------------------------------------------------------------------------------------------------------------------------------------------
# 画面の文字表示　各frameへ配置　センター配置ならpack　他はplace(各frameに対して)

info = tk.Label(frame1, text="tab変換ファイル結合は新しい名前で保存", font=fonts, pady=5)
info.pack(anchor='center')

info_name = tk.Label(frame2, text="保存ファイル名コピー用", font=fonts)
info_name.place(x=15, y=0)

wr_filename = tk.Entry(frame2, width=27, font=fonts)
wr_filename.place(x=15, y=25, height=20)

info_header = tk.Label(frame4, text="保存オプション")
info_header.place(x=20, y=5)


root.mainloop()

'utf-8' codec can't decode byte 0x91 in position 0: invalid start byte from_csv_convert
保存ファイル名：000_2020_clg_Fielders-sjis.xlsx
'utf-8' codec can't decode byte 0x91 in position 0: invalid start byte from_csv_convert
保存ファイル名：000_2020_clg_Pitchers-sjis.xlsx
'utf-8' codec can't decode byte 0x91 in position 0: invalid start byte from_csv2xlsx_tab
'utf-8' codec can't decode byte 0x91 in position 0: invalid start byte from_csv2xlsx_tab
'utf-8' codec can't decode byte 0x91 in position 0: invalid start byte from_csv2xlsx_tab
'utf-8' codec can't decode byte 0x91 in position 0: invalid start byte from_csv2xlsx_tab
保存ファイル名：off.xlsx
'utf-8' codec can't decode byte 0x91 in position 0: invalid start byte from_csv2xlsx_tab
'utf-8' codec can't decode byte 0x91 in position 0: invalid start byte from_csv2xlsx_tab
'utf-8' codec can't decode byte 0x91 in position 0: invalid start byte from_csv2xlsx_tab
'utf-8' codec can't decode byte 0x91 in position 0: invalid start byte from_csv2xlsx_tab
保存ファイル名：on.xlsx